In [9]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# What version of Python do you have?
import torch
import sklearn as sk
from sklearn.model_selection import StratifiedKFold
import copy
print(f"PyTorch Version: {torch.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if torch.cuda.is_available() else "NOT AVAILABLE")

PyTorch Version: 1.7.0

Python 3.8.6 (default, Oct  8 2020, 14:06:32) 
[Clang 12.0.0 (clang-1200.0.32.2)]
Pandas 1.1.2
Scikit-Learn 0.23.2
GPU is NOT AVAILABLE


# Normalizar Datos

In [10]:
def normalize(vector):
    media_vector = vector.mean(axis=0)
    std_vector = vector.std(axis=0)
    return (vector - media_vector)/std_vector, media_vector, std_vector

# Sigmoidal

In [11]:
# implemento la función sigmoidal
def sigma(x):
    return 1.0 / (1 + torch.exp(-x)) 


A = torch.tensor([[1.,0.],[0.,1.]]).to('cuda')
b = torch.tensor([[0.],[0.]]).to('cuda')
ans = torch.matmul(A,b)
sigma(ans)

AssertionError: Torch not compiled with CUDA enabled

# Calcular Función de Costo
En teoría la función costo es la norma del vector resultante de la diferencia del valor resultante de la entrada X y su valor resultante definido.  
$$
Cost(X, y) = \sum_{i=0}^{m}(y^i- \hat{y}^i)^{2} = \|h_{\theta}(X) - y\|
$$
En el caso de nuestra implementación como todos los datos de entrada serán insertados en una sola pasada, necesito el promedio de la diferencia de cada valor de salida.  

In [41]:
# recibo los datos
def Calcular_Funcion_Costo(y):
    return torch.linalg.norm(y)
    #return torch.linalg.norm(y) / y.nelement()
    #return torch.sqrt(torch.sum(y**2)) / y.nelement()
    #return torch.mean(torch.linalg.norm(vector, dim=0))
A = torch.tensor([[1.,0.],[0.,1.]]).to('cuda')
b = torch.tensor([[0.],[0.]]).to('cuda')
ans = torch.matmul(A,b)
Calcular_Funcion_Costo(sigma(ans))

tensor(0.7071, device='cuda:0')

# dS
Recibe los datos $D$ y calcula para dichos datos la derivada de la función sigmoidal.  
Recibiré un vector $a$ como en la teoría está establecido, luego retornaré la derivada de la función sigmoidal, dada por la ecuación:  
$$
\frac{d\sigma(x)}{dx} = \sigma(x)(1 - \sigma(x))
$$
Todo esto en notación vectorial.

In [42]:
def dS(D):
    # recordemos que la variable sigma es la función sigmoidal vectorizada
    return sigma(D)*(1 - sigma(D))

a = torch.tensor([[1.],[2.]]).to('cuda')
print(dS(a))

tensor([[0.1966],
        [0.1050]], device='cuda:0')


# Forward
Recibe los datos $X$ y un diccionario de parámetros $W$ (i.e. los pesos de cada capa), donde cada elemento del diccionario es una matriz de pesos; y realiza la etapa de propagación. Devuelve un diccionario de activaciones $A$, donde cada elemento del diccionario son las activaciones de cada capa.

In [43]:
# El primer paso es crear todas las matrices de pesos basándonos en las dimensiones de la entrada X
# y el tamaño del batch que ingresaremos
def generar_pesos(parameters):
    layers = {}
    n = len(parameters)
    # i -> (0,1,...,n-1)
    for i in range(0, n-1):
        layers[i] = {"W": torch.randn(parameters[i+1], parameters[i], dtype=torch.double).to('cuda'),
                     "b": torch.randn(parameters[i+1], 1, dtype=torch.double).to('cuda')}
    return layers

def Forward(X, W):
    A = {0: {"a": X, "z": X}} # el diccionario a retornar
               # como primer elemento de la lista esta X, para mantener el orden
    n = len(W) # primero extraigo la cantidad de pesos para iterar sobre eso
    for i in range(1, n + 1):
        activation = torch.matmul(W[i-1]["W"], A[i-1]["a"]) + W[i-1]["b"] # se hace broadcasting al sumar el bias
        A[i] = {"a": sigma(activation), "z": activation}
    return A


Wout = generar_pesos([3,2,3])
print(Wout)
Xin = torch.tensor([[100, 10],[50, 70],[74, 20]], dtype=torch.double).to('cuda')
print(Forward(Xin, Wout))

{0: {'W': tensor([[-0.5018,  0.9151,  0.4637],
        [ 0.3624, -1.0123, -0.5464]], device='cuda:0', dtype=torch.float64), 'b': tensor([[-1.0648],
        [-0.0363]], device='cuda:0', dtype=torch.float64)}, 1: {'W': tensor([[-0.3462,  0.5672],
        [ 0.5015,  0.2719],
        [-2.1228,  0.5674]], device='cuda:0', dtype=torch.float64), 'b': tensor([[-0.2541],
        [-0.5318],
        [ 1.6055]], device='cuda:0', dtype=torch.float64)}}
{0: {'a': tensor([[100.,  10.],
        [ 50.,  70.],
        [ 74.,  20.]], device='cuda:0', dtype=torch.float64), 'z': tensor([[100.,  10.],
        [ 50.,  70.],
        [ 74.,  20.]], device='cuda:0', dtype=torch.float64)}, 1: {'a': tensor([[1.0000e+00, 1.0000e+00],
        [1.5216e-24, 1.0899e-34]], device='cuda:0', dtype=torch.float64), 'z': tensor([[ 28.8280,  67.2481],
        [-54.8423, -78.2018]], device='cuda:0', dtype=torch.float64)}, 2: {'a': tensor([[0.3543, 0.3543],
        [0.4924, 0.4924],
        [0.3735, 0.3735]], device='cuda:0', 

# Backward

In [44]:
def Backward(X, A, W, y, lr):
    """
    X: valores de entrada
    A: diccionario con los resultados de cada capa
    W: diccionario con los pesos de cada capa
    y: salida de la última capa
    lr: tasa de aprendizaje (learning rate en inglés)
    """
    r = len(A)-1
    deltas = {r: -(y - A[r]["a"]) * dS(A[r]["z"])}
    for i in range(r-1,-1,-1):
        deltas[i] = torch.matmul(torch.t(W[i]["W"]), deltas[i+1]) * dS(A[i]["z"])
        # ahora viene la etapa del gradiente descendiente, para los pesos
        W[i]["W"] = W[i]["W"] - lr * torch.matmul(deltas[i+1], torch.t(A[i]["a"]))
        W[i]["b"] = W[i]["b"] - lr * deltas[i+1]
        W[i]["b"] = torch.mean(W[i]["b"], dim=1, keepdim=True)
    return W

Wout = generar_pesos([3,2,3])
print(Wout)
Xin = torch.tensor([[100, 10],[50, 70],[74, 20]], dtype=torch.double).to('cuda')
A = Forward(Xin, Wout)
yout = torch.tensor([[1, 0],[0, 1],[0, 0]], dtype=torch.double).to('cuda')
print(Backward(Xin, A, Wout, yout, 0.5))

{0: {'W': tensor([[-0.3144, -0.7152,  1.7637],
        [-0.8687, -1.0222, -0.7393]], device='cuda:0', dtype=torch.float64), 'b': tensor([[ 0.0831],
        [-1.6285]], device='cuda:0', dtype=torch.float64)}, 1: {'W': tensor([[-0.8095, -0.1356],
        [-1.2258,  0.7945],
        [ 1.8109,  0.0851]], device='cuda:0', dtype=torch.float64), 'b': tensor([[ 0.9202],
        [-0.6306],
        [ 0.2565]], device='cuda:0', dtype=torch.float64)}}
{0: {'W': tensor([[-0.3144, -0.7152,  1.7637],
        [-0.8687, -1.0222, -0.7393]], device='cuda:0', dtype=torch.float64), 'b': tensor([[ 0.0831],
        [-1.6285]], device='cuda:0', dtype=torch.float64)}, 1: {'W': tensor([[-0.7506, -0.1356],
        [-1.2337,  0.7945],
        [ 1.7666,  0.0851]], device='cuda:0', dtype=torch.float64), 'b': tensor([[ 0.9132],
        [-0.5976],
        [ 0.1997]], device='cuda:0', dtype=torch.float64)}}


# Gradiente Descendiente

In [52]:
def Gradiente_Descendiente(X, y, W, epochs, lr):
    result = {"costs": []}
    Wout = copy.deepcopy(W)
    for i in range(epochs):
        Aout = Forward(X, Wout)
        #result["weigths"].append(W)
        result["costs"].append(Calcular_Funcion_Costo(y - Aout[len(Aout)-1]["a"]))
        Backward(X, Aout, Wout, y, lr)
    
    #for idx in Wout:
        #Wout[idx]['b'] = Wout[idx]['b'][:,0:1]
    result['W'] = Wout
    return result

Wout = generar_pesos([3,4,3])
Xin = torch.tensor([[100, 10],[50, 100],[74, 200]], dtype=torch.double).to('cuda')
yout = torch.tensor([[1, 0],[0, 1],[0, 0]], dtype=torch.double).to('cuda')

ans = Gradiente_Descendiente(Xin, yout, Wout, 2000, 0.1)
print(Xin)
print(yout)
print(Forward(Xin, ans['W'])[2]["a"])
print(ans['W'])
ans["costs"][-4:-1]

tensor([[100.,  10.],
        [ 50., 100.],
        [ 74., 200.]], device='cuda:0', dtype=torch.float64)
tensor([[1., 0.],
        [0., 1.],
        [0., 0.]], device='cuda:0', dtype=torch.float64)
tensor([[0.9173, 0.0771],
        [0.0870, 0.9189],
        [0.0223, 0.0110]], device='cuda:0', dtype=torch.float64)
{0: {'W': tensor([[ 0.7652,  0.4212,  0.3007],
        [ 0.3215,  0.2752,  1.5116],
        [-1.5492, -0.1839,  0.6061],
        [ 1.4598, -0.7738,  0.4441]], device='cuda:0', dtype=torch.float64), 'b': tensor([[ 0.9583],
        [-0.2309],
        [ 1.7836],
        [ 0.0221]], device='cuda:0', dtype=torch.float64)}, 1: {'W': tensor([[ 0.9531,  0.4864, -4.8897,  1.3988],
        [-0.8710, -1.3368,  4.7782,  0.4496],
        [-0.5446, -1.0077, -0.7202, -0.6816]], device='cuda:0',
       dtype=torch.float64), 'b': tensor([[-0.4315],
        [-0.5926],
        [-1.5451]], device='cuda:0', dtype=torch.float64)}}


[tensor(0.1661, device='cuda:0', dtype=torch.float64),
 tensor(0.1661, device='cuda:0', dtype=torch.float64),
 tensor(0.1660, device='cuda:0', dtype=torch.float64)]

In [53]:
# estimaremos el resultado de nuestros vectores considerando la posición del vector 
#con la máxima puntuación como 1 y el resto como 0, p.ej.
# si tueviésemos un vector a = [0.5, 0.2, 0.7], su estimado será [0, 0, 1]
def estimate_result(x): # receives a vector of n*1
    Xout = torch.zeros(x.shape[0], x.shape[1], dtype=torch.int).to('cuda')
    maxidx = torch.argmax(x, dim=0)
    for i in range(maxidx.shape[0]):
        Xout[maxidx[i], i] = 1
    return Xout

# una pequeña prueba
vectest = torch.tensor([[0.5],[0.2],[0.7]], dtype=torch.double).to('cuda')
vectest = estimate_result(vectest)
vectest

tensor([[0],
        [0],
        [1]], device='cuda:0', dtype=torch.int32)

# Calcular Accuracy

In [54]:
def Calcular_Accuracy(YMLP, Y):
    set_size = YMLP.shape[1]
    correct = 0.0
    for i in range(set_size):
        if torch.all(torch.eq(YMLP[:,i], Y[:,i])):
            correct += 1.0
    return correct / set_size

YMLP = torch.tensor([[1,0],[0,0],[0,1]], dtype=torch.double).to('cuda')
Y = torch.tensor([[1,0],[1,0],[0,1]], dtype=torch.double).to('cuda')
Calcular_Accuracy(YMLP, Y)

0.5

# K-fold Cross Validation

In [55]:
# myset será una lista con los valores de los 
def KfoldsCrossValidation(X, y, Y, params, k=3, shuff=True):
    """
    X: el conjunto de datos reescalados
    y: la columna de datos categóricos
    Y: la columna de los datos categóricos con la configuración one hot
    params: un diccionario de datos con los siguiente parámetros
            -layers: una lista con el número de neuronas por cada capa
            -learning: una lista con las tasas de aprendizaje a evaluar
            -epochs: una lista con el número de épocas(iteraciones)
    k: el k para realizar el k-folds (3 por defecto)
    shuff: la variable booleana para decidir si los datos se barajan o no
    """
    skf = StratifiedKFold(n_splits=k, shuffle=shuff)
    learning_rates = params['learning']
    epochs = params['epochs']
    layers = params['layers']
    bestAcc = 0
    bestLr = -1
    bestEpoch = -1
    Weights = generar_pesos(layers)
    for lr in learning_rates:
        for epoch in epochs:
            print(f'Tasa de aprendizaje: {lr}, épocas: {epoch} ', end="")
            avgAcc = 0
            for train_index, test_index in skf.split(X, y):
                #print("TRAIN:", train_index[0:10], "TEST:", test_index[0:10])
                X_train, X_test = torch.from_numpy(np.array(X[train_index], dtype='double').T).to('cuda'), torch.from_numpy(np.array(X[test_index], dtype='double').T).to('cuda')
                y_train, y_test = torch.from_numpy(np.array(Y[train_index], dtype='double').T).to('cuda'), torch.from_numpy(np.array(Y[test_index], dtype='int').T).to('cuda')
                ans = Gradiente_Descendiente(X_train, y_train, Weights, epoch, lr)
                acc = estimate_result(Forward(X_test, ans['W'])[len(layers)-1]["a"]) # primero estimamos el valor de la última capa
                acc = Calcular_Accuracy(acc, y_test)
                avgAcc += acc
            avgAcc /= k
            if avgAcc > bestAcc:
                bestAcc = avgAcc
                bestLr = lr
                bestEpoch = epoch
            print(f'Average acc: {avgAcc}')
            
                
    print(f'Results\nBest learning rate: {bestLr}')
    print(f'Best Nro epochs: {bestEpoch}')
    print(f'Best Average Accuracy: {bestAcc}')
        

# Clasificación del género de música

In [43]:
# leo los datos
Data = pd.read_csv('music_genre.csv')
Data = Data.drop(['filename'], axis=1)
# hacemos un shuffle de los datos
# Data = Data.sample(frac=1).reset_index(drop=True)
print("Music genres:\n", Data['label'].value_counts())
Data.head()

Music genres:
 classical    100
disco        100
reggae       100
hiphop       100
rock         100
blues        100
country      100
pop          100
jazz         100
metal        100
Name: label, dtype: int64


,tempo,beats,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,103.359375,50,0.380260,0.248262,2116.942959,1956.611056,4196.107960,0.127272,-26.929785,107.334008,...,14.336612,-13.821769,7.562789,-6.181372,0.330165,-6.829571,0.965922,-7.570825,2.918987,blues
1,95.703125,44,0.306451,0.113475,1156.070496,1497.668176,2170.053545,0.058613,-233.860772,136.170239,...,-2.250578,3.959198,5.322555,0.812028,-1.107202,-4.556555,-2.436490,3.316913,-0.608485,blues
2,151.999081,75,0.253487,0.151571,1331.073970,1973.643437,2900.174130,0.042967,-221.802549,110.843070,...,-13.037723,-12.652228,-1.821905,-7.260097,-6.660252,-14.682694,-11.719264,-11.025216,-13.387260,blues
3,184.570312,91,0.269320,0.119072,1361.045467,1567.804596,2739.625101,0.069124,-207.208080,132.799175,...,-0.613248,0.384877,2.605128,-5.188924,-9.527455,-9.244394,-2.848274,-1.418707,-5.932607,blues
4,161.499023,74,0.391059,0.137728,1811.076084,2052.332563,3927.809582,0.075480,-145.434568,102.829023,...,7.457218,-10.470444,-2.360483,-6.783623,2.671134,-4.760879,-0.949005,0.024832,-2.005315,blues


## One hot encoding para los datos

In [44]:
genres = pd.get_dummies(Data[['label']])
Xinput = Data.drop(['label'], axis=1)

In [45]:
# aplicando la normalización de los datos
Xinput, mediaXinput, stdXinput = normalize(Xinput)
Data.head()

,tempo,beats,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,103.359375,50,0.380260,0.248262,2116.942959,1956.611056,4196.107960,0.127272,-26.929785,107.334008,...,14.336612,-13.821769,7.562789,-6.181372,0.330165,-6.829571,0.965922,-7.570825,2.918987,blues
1,95.703125,44,0.306451,0.113475,1156.070496,1497.668176,2170.053545,0.058613,-233.860772,136.170239,...,-2.250578,3.959198,5.322555,0.812028,-1.107202,-4.556555,-2.436490,3.316913,-0.608485,blues
2,151.999081,75,0.253487,0.151571,1331.073970,1973.643437,2900.174130,0.042967,-221.802549,110.843070,...,-13.037723,-12.652228,-1.821905,-7.260097,-6.660252,-14.682694,-11.719264,-11.025216,-13.387260,blues
3,184.570312,91,0.269320,0.119072,1361.045467,1567.804596,2739.625101,0.069124,-207.208080,132.799175,...,-0.613248,0.384877,2.605128,-5.188924,-9.527455,-9.244394,-2.848274,-1.418707,-5.932607,blues
4,161.499023,74,0.391059,0.137728,1811.076084,2052.332563,3927.809582,0.075480,-145.434568,102.829023,...,7.457218,-10.470444,-2.360483,-6.783623,2.671134,-4.760879,-0.949005,0.024832,-2.005315,blues


In [46]:
print(Xinput.shape)
print(genres.shape)

(1000, 28)
(1000, 10)


## K-folds cross validation
### 0 capas intermedias

In [166]:
myparams = {
            'layers': [Xinput.shape[1],genres.shape[1]],
            'epochs': [1000, 1500, 2000, 2500, 3000],
            'learning': [0.2, 0.1, 0.07, 0.05]
           }
KfoldsCrossValidation(np.array(Xinput), np.array(Data['label']), np.array(genres), myparams, 3, True)

Tasa de aprendizaje: 0.2, épocas: 1000 Average acc: 0.2580334825843808
Tasa de aprendizaje: 0.2, épocas: 1500 Average acc: 0.30398362434290577
Tasa de aprendizaje: 0.2, épocas: 2000 Average acc: 0.3069926213638788
Tasa de aprendizaje: 0.2, épocas: 2500 Average acc: 0.3109936283588978
Tasa de aprendizaje: 0.2, épocas: 3000 Average acc: 0.2929995864127601
Tasa de aprendizaje: 0.1, épocas: 1000 Average acc: 0.3179616742490994
Tasa de aprendizaje: 0.1, épocas: 1500 Average acc: 0.3060185934437431
Tasa de aprendizaje: 0.1, épocas: 2000 Average acc: 0.3369687052321783
Tasa de aprendizaje: 0.1, épocas: 2500 Average acc: 0.28702055348761935
Tasa de aprendizaje: 0.1, épocas: 3000 Average acc: 0.29296362230493966
Tasa de aprendizaje: 0.07, épocas: 1000 Average acc: 0.3299826772880665
Tasa de aprendizaje: 0.07, épocas: 1500 Average acc: 0.3249686812561064
Tasa de aprendizaje: 0.07, épocas: 2000 Average acc: 0.3059526592460724
Tasa de aprendizaje: 0.07, épocas: 2500 Average acc: 0.3180036323748899

### 1 capa intermedia

In [168]:
myparams['layers'] = [Xinput.shape[1],10,genres.shape[1]]
KfoldsCrossValidation(np.array(Xinput), np.array(Data['label']), np.array(genres), myparams, 3, True)

Tasa de aprendizaje: 0.2, épocas: 1000 Average acc: 0.2600055144965325
Tasa de aprendizaje: 0.2, épocas: 1500 Average acc: 0.2750145354935774
Tasa de aprendizaje: 0.2, épocas: 2000 Average acc: 0.31198563833294374
Tasa de aprendizaje: 0.2, épocas: 2500 Average acc: 0.2720085354815894
Tasa de aprendizaje: 0.2, épocas: 3000 Average acc: 0.27299754844665025
Tasa de aprendizaje: 0.1, épocas: 1000 Average acc: 0.46094897292502085
Tasa de aprendizaje: 0.1, épocas: 1500 Average acc: 0.4319589050127972
Tasa de aprendizaje: 0.1, épocas: 2000 Average acc: 0.4569419719120318
Tasa de aprendizaje: 0.1, épocas: 2500 Average acc: 0.43898389407371446
Tasa de aprendizaje: 0.1, épocas: 3000 Average acc: 0.4539689389988791
Tasa de aprendizaje: 0.07, épocas: 1000 Average acc: 0.5659881438324552
Tasa de aprendizaje: 0.07, épocas: 1500 Average acc: 0.5620141099183016
Tasa de aprendizaje: 0.07, épocas: 2000 Average acc: 0.5740051428674183
Tasa de aprendizaje: 0.07, épocas: 2500 Average acc: 0.549960139780499

### 2 capas intermedias

In [162]:
myparams['layers'] = [Xinput.shape[1],10,10,genres.shape[1]]
KfoldsCrossValidation(np.array(Xinput), np.array(Data['label']), np.array(genres), myparams, 3, True)

Tasa de aprendizaje: 0.2, épocas: 1000 Average acc: 0.26702750654846463
Tasa de aprendizaje: 0.2, épocas: 1500 Average acc: 0.24694454933975893
Tasa de aprendizaje: 0.2, épocas: 2000 Average acc: 0.3059976143808479
Tasa de aprendizaje: 0.2, épocas: 2500 Average acc: 0.2560164955374536
Tasa de aprendizaje: 0.2, épocas: 3000 Average acc: 0.25301648954343564
Tasa de aprendizaje: 0.1, épocas: 1000 Average acc: 0.3280016543489597
Tasa de aprendizaje: 0.1, épocas: 1500 Average acc: 0.34204264144383906
Tasa de aprendizaje: 0.1, épocas: 2000 Average acc: 0.34893576210941485
Tasa de aprendizaje: 0.1, épocas: 2500 Average acc: 0.36602171033308756
Tasa de aprendizaje: 0.1, épocas: 3000 Average acc: 0.32398865931800064
Tasa de aprendizaje: 0.07, épocas: 1000 Average acc: 0.3710087332841823
Tasa de aprendizaje: 0.07, épocas: 1500 Average acc: 0.3819717921514329
Tasa de aprendizaje: 0.07, épocas: 2000 Average acc: 0.384986783190376
Tasa de aprendizaje: 0.07, épocas: 2500 Average acc: 0.3690277103450

### 3 capas intermedias

In [163]:
myparams['layers'] = [Xinput.shape[1],10, 10, 10, genres.shape[1]]
KfoldsCrossValidation(np.array(Xinput), np.array(Data['label']), np.array(genres), myparams, 3, True)

Tasa de aprendizaje: 0.2, épocas: 1000 Average acc: 0.1289463115810421
Tasa de aprendizaje: 0.2, épocas: 1500 Average acc: 0.10800620980261699
Tasa de aprendizaje: 0.2, épocas: 2000 Average acc: 0.10099920279560998
Tasa de aprendizaje: 0.2, épocas: 2500 Average acc: 0.14003824183464902
Tasa de aprendizaje: 0.2, épocas: 3000 Average acc: 0.130936325547104
Tasa de aprendizaje: 0.1, épocas: 1000 Average acc: 0.4450228671785558
Tasa de aprendizaje: 0.1, épocas: 1500 Average acc: 0.4609789430148712
Tasa de aprendizaje: 0.1, épocas: 2000 Average acc: 0.46699094303884725
Tasa de aprendizaje: 0.1, épocas: 2500 Average acc: 0.44404284524045007
Tasa de aprendizaje: 0.1, épocas: 3000 Average acc: 0.45800291309273344
Tasa de aprendizaje: 0.07, épocas: 1000 Average acc: 0.43901086715457965
Tasa de aprendizaje: 0.07, épocas: 1500 Average acc: 0.45898892904880934
Tasa de aprendizaje: 0.07, épocas: 2000 Average acc: 0.4519789250328172
Tasa de aprendizaje: 0.07, épocas: 2500 Average acc: 0.457996919074

### 4 capas intermedias

In [165]:
myparams['layers'] = [Xinput.shape[1], 10, 10, 10, 10, genres.shape[1]]
KfoldsCrossValidation(np.array(Xinput), np.array(Data['label']), np.array(genres), myparams, 3, True)

Tasa de aprendizaje: 0.2, épocas: 1000 Average acc: 0.10000119880359402
Tasa de aprendizaje: 0.2, épocas: 1500 Average acc: 0.10099920279560998
Tasa de aprendizaje: 0.2, épocas: 2000 Average acc: 0.10099920279560999
Tasa de aprendizaje: 0.2, épocas: 2500 Average acc: 0.10100219980459502
Tasa de aprendizaje: 0.2, épocas: 3000 Average acc: 0.099000197802593
Tasa de aprendizaje: 0.1, épocas: 1000 Average acc: 0.4219908531285777
Tasa de aprendizaje: 0.1, épocas: 1500 Average acc: 0.3089916263568958
Tasa de aprendizaje: 0.1, épocas: 2000 Average acc: 0.3969568370765975
Tasa de aprendizaje: 0.1, épocas: 2500 Average acc: 0.4290068511625398
Tasa de aprendizaje: 0.1, épocas: 3000 Average acc: 0.334034633435831
Tasa de aprendizaje: 0.07, épocas: 1000 Average acc: 0.5169990349631068
Tasa de aprendizaje: 0.07, épocas: 1500 Average acc: 0.44197490904077724
Tasa de aprendizaje: 0.07, épocas: 2000 Average acc: 0.45299491107874346
Tasa de aprendizaje: 0.07, épocas: 2500 Average acc: 0.466972960984937

# Clasificación Titanic

In [12]:
# leo los datos
TitanicData = pd.read_csv('titanic_train.csv')
TitanicData = TitanicData[['Pclass', 'Survived', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
# convertimos a strings las siguientes columnas para hacer el OneHot encoding
TitanicData['Survived'] = TitanicData['Survived'].apply(str)
TitanicData['Pclass'] = TitanicData['Pclass'].apply(str)
age_mean = TitanicData["Age"].mean()
TitanicData['Age'].fillna(age_mean, inplace = True)
TitanicData['Embarked'].fillna('S', inplace = True)
# Passenger class(Pclass): 1 = 1st, 2 = 2nd, 3 = 3rd
# Puertos de embarque(Embarked): C = Cherbourg, Q = Queenstown, S = Southampton
# Sexo(Sex): Male, Female
TitanicData = pd.get_dummies(TitanicData, columns=['Sex', 'Embarked', 'Pclass'])
TitanicData.info()
#print(TitanicData['SibSp'].value_counts())
#print(TitanicData['Parch'].value_counts())
#print(TitanicData['Embarked'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    object 
 1   Age         891 non-null    float64
 2   SibSp       891 non-null    int64  
 3   Parch       891 non-null    int64  
 4   Fare        891 non-null    float64
 5   Sex_female  891 non-null    uint8  
 6   Sex_male    891 non-null    uint8  
 7   Embarked_C  891 non-null    uint8  
 8   Embarked_Q  891 non-null    uint8  
 9   Embarked_S  891 non-null    uint8  
 10  Pclass_1    891 non-null    uint8  
 11  Pclass_2    891 non-null    uint8  
 12  Pclass_3    891 non-null    uint8  
dtypes: float64(2), int64(2), object(1), uint8(8)
memory usage: 41.9+ KB


# Preprocesamiento de la data

In [13]:
# convertiremos el campo Survived a codificación one hot, ya que será el valor para nuestra clasificación
# Survived: 0 = No, 1 = Yes
survived = pd.get_dummies(TitanicData['Survived'])
Xinput = TitanicData.drop(['Survived'], axis=1)
Xinput.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Age         891 non-null    float64
 1   SibSp       891 non-null    int64  
 2   Parch       891 non-null    int64  
 3   Fare        891 non-null    float64
 4   Sex_female  891 non-null    uint8  
 5   Sex_male    891 non-null    uint8  
 6   Embarked_C  891 non-null    uint8  
 7   Embarked_Q  891 non-null    uint8  
 8   Embarked_S  891 non-null    uint8  
 9   Pclass_1    891 non-null    uint8  
 10  Pclass_2    891 non-null    uint8  
 11  Pclass_3    891 non-null    uint8  
dtypes: float64(2), int64(2), uint8(8)
memory usage: 34.9 KB


In [14]:
# aplicando la normalización de los datos
Xinput, mediaXinput, stdXinput = normalize(Xinput)
Xinput.head()

,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,-0.592148,0.432550,-0.473408,-0.502163,-0.737281,0.737281,-0.481772,-0.30739,0.615493,-0.565368,-0.509865,0.902081
1,0.638430,0.432550,-0.473408,0.786404,1.354813,-1.354813,2.073341,-0.30739,-1.622891,1.766775,-0.509865,-1.107304
2,-0.284503,-0.474279,-0.473408,-0.488580,1.354813,-1.354813,-0.481772,-0.30739,0.615493,-0.565368,-0.509865,0.902081
3,0.407697,0.432550,-0.473408,0.420494,1.354813,-1.354813,-0.481772,-0.30739,0.615493,1.766775,-0.509865,-1.107304
4,0.407697,-0.474279,-0.473408,-0.486064,-0.737281,0.737281,-0.481772,-0.30739,0.615493,-0.565368,-0.509865,0.902081


In [15]:
print(Xinput.shape)
print(survived.shape)


(891, 12)
(891, 2)


## K-folds cross validation
### 0 capas intermedias

In [16]:
myparams = {
            'layers': [Xinput.shape[1],survived.shape[1]],
            'epochs': [1000, 1500, 2000, 2500, 3000],
            'learning': [0.2, 0.1, 0.07, 0.05]
           }
KfoldsCrossValidation(np.array(Xinput), np.array(TitanicData['Survived']), np.array(survived), myparams, 3, True)

NameError: name 'KfoldsCrossValidation' is not defined

### 1 capa intermedia

In [57]:
myparams['layers'] = [Xinput.shape[1], 10,survived.shape[1]]
KfoldsCrossValidation(np.array(Xinput), np.array(TitanicData['Survived']), np.array(survived), myparams, 3, True)

Tasa de aprendizaje: 0.2, épocas: 1000 Average acc: 0.7878787878787877
Tasa de aprendizaje: 0.2, épocas: 1500 Average acc: 0.7833894500561168
Tasa de aprendizaje: 0.2, épocas: 2000 Average acc: 0.7822671156004489
Tasa de aprendizaje: 0.2, épocas: 2500 Average acc: 0.7946127946127945
Tasa de aprendizaje: 0.2, épocas: 3000 Average acc: 0.7744107744107743
Tasa de aprendizaje: 0.1, épocas: 1000 Average acc: 0.7744107744107743
Tasa de aprendizaje: 0.1, épocas: 1500 Average acc: 0.7912457912457912
Tasa de aprendizaje: 0.1, épocas: 2000 Average acc: 0.7699214365881032
Tasa de aprendizaje: 0.1, épocas: 2500 Average acc: 0.7901234567901235
Tasa de aprendizaje: 0.1, épocas: 3000 Average acc: 0.7968574635241302
Tasa de aprendizaje: 0.07, épocas: 1000 Average acc: 0.7845117845117845
Tasa de aprendizaje: 0.07, épocas: 1500 Average acc: 0.8058361391694725
Tasa de aprendizaje: 0.07, épocas: 2000 Average acc: 0.787878787878788
Tasa de aprendizaje: 0.07, épocas: 2500 Average acc: 0.7901234567901234
Tas

### 2 capas intermedias 

In [58]:
myparams['layers'] = [Xinput.shape[1], 10, 10,survived.shape[1]]
KfoldsCrossValidation(np.array(Xinput), np.array(TitanicData['Survived']), np.array(survived), myparams, 3, True)

Tasa de aprendizaje: 0.2, épocas: 1000 Average acc: 0.7777777777777778
Tasa de aprendizaje: 0.2, épocas: 1500 Average acc: 0.7699214365881032
Tasa de aprendizaje: 0.2, épocas: 2000 Average acc: 0.7631874298540966
Tasa de aprendizaje: 0.2, épocas: 2500 Average acc: 0.7923681257014591
Tasa de aprendizaje: 0.2, épocas: 3000 Average acc: 0.7912457912457912
Tasa de aprendizaje: 0.1, épocas: 1000 Average acc: 0.8114478114478114
Tasa de aprendizaje: 0.1, épocas: 1500 Average acc: 0.7822671156004489
Tasa de aprendizaje: 0.1, épocas: 2000 Average acc: 0.7710437710437711
Tasa de aprendizaje: 0.1, épocas: 2500 Average acc: 0.7609427609427609
Tasa de aprendizaje: 0.1, épocas: 3000 Average acc: 0.7856341189674523
Tasa de aprendizaje: 0.07, épocas: 1000 Average acc: 0.7845117845117846
Tasa de aprendizaje: 0.07, épocas: 1500 Average acc: 0.7744107744107743
Tasa de aprendizaje: 0.07, épocas: 2000 Average acc: 0.7833894500561168
Tasa de aprendizaje: 0.07, épocas: 2500 Average acc: 0.7800224466891134
Ta

### 3 capas intermedias 

In [59]:
myparams['layers'] = [Xinput.shape[1], 10, 10, 10, survived.shape[1]]
KfoldsCrossValidation(np.array(Xinput), np.array(TitanicData['Survived']), np.array(survived), myparams, 3, True)

Tasa de aprendizaje: 0.2, épocas: 1000 Average acc: 0.3838383838383838
Tasa de aprendizaje: 0.2, épocas: 1500 Average acc: 0.8080808080808081
Tasa de aprendizaje: 0.2, épocas: 2000 Average acc: 0.7811447811447811
Tasa de aprendizaje: 0.2, épocas: 2500 Average acc: 0.7923681257014591
Tasa de aprendizaje: 0.2, épocas: 3000 Average acc: 0.7979797979797979
Tasa de aprendizaje: 0.1, épocas: 1000 Average acc: 0.7878787878787877
Tasa de aprendizaje: 0.1, épocas: 1500 Average acc: 0.7811447811447811
Tasa de aprendizaje: 0.1, épocas: 2000 Average acc: 0.7923681257014591
Tasa de aprendizaje: 0.1, épocas: 2500 Average acc: 0.7822671156004489
Tasa de aprendizaje: 0.1, épocas: 3000 Average acc: 0.7755331088664422
Tasa de aprendizaje: 0.07, épocas: 1000 Average acc: 0.7934904601571269
Tasa de aprendizaje: 0.07, épocas: 1500 Average acc: 0.7912457912457912
Tasa de aprendizaje: 0.07, épocas: 2000 Average acc: 0.7732884399551067
Tasa de aprendizaje: 0.07, épocas: 2500 Average acc: 0.7800224466891134
Ta

### 4 capas intermedias

In [60]:
myparams['layers'] = [Xinput.shape[1], 10, 10, 10, 10, survived.shape[1]]
KfoldsCrossValidation(np.array(Xinput), np.array(TitanicData['Survived']), np.array(survived), myparams, 3, True)

Tasa de aprendizaje: 0.2, épocas: 1000 Average acc: 0.6161616161616161
Tasa de aprendizaje: 0.2, épocas: 1500 Average acc: 0.7272727272727272
Tasa de aprendizaje: 0.2, épocas: 2000 Average acc: 0.7811447811447811
Tasa de aprendizaje: 0.2, épocas: 2500 Average acc: 0.734006734006734
Tasa de aprendizaje: 0.2, épocas: 3000 Average acc: 0.6711560044893378
Tasa de aprendizaje: 0.1, épocas: 1000 Average acc: 0.8002244668911335
Tasa de aprendizaje: 0.1, épocas: 1500 Average acc: 0.7934904601571269
Tasa de aprendizaje: 0.1, épocas: 2000 Average acc: 0.7789001122334455
Tasa de aprendizaje: 0.1, épocas: 2500 Average acc: 0.7822671156004489
Tasa de aprendizaje: 0.1, épocas: 3000 Average acc: 0.7845117845117845
Tasa de aprendizaje: 0.07, épocas: 1000 Average acc: 0.7934904601571269
Tasa de aprendizaje: 0.07, épocas: 1500 Average acc: 0.7800224466891134
Tasa de aprendizaje: 0.07, épocas: 2000 Average acc: 0.7833894500561168
Tasa de aprendizaje: 0.07, épocas: 2500 Average acc: 0.7946127946127947
Tas

# SVM

In [38]:
from sklearn import svm
from sklearn.metrics import mean_squared_error, r2_score

In [39]:
# leo los datos
TitanicData = pd.read_csv('titanic_train.csv')
TitanicData = TitanicData[['Pclass', 'Survived', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
# convertimos a strings las siguientes columnas para hacer el OneHot encoding
TitanicData['Survived'] = TitanicData['Survived'].apply(str)
TitanicData['Pclass'] = TitanicData['Pclass'].apply(str)
age_mean = TitanicData["Age"].mean()
TitanicData['Age'].fillna(age_mean, inplace = True)
TitanicData['Embarked'].fillna('S', inplace = True)
# Passenger class(Pclass): 1 = 1st, 2 = 2nd, 3 = 3rd
# Puertos de embarque(Embarked): C = Cherbourg, Q = Queenstown, S = Southampton
# Sexo(Sex): Male, Female
TitanicData = pd.get_dummies(TitanicData, columns=['Sex', 'Embarked', 'Pclass'])
TitanicData.info()
#print(TitanicData['SibSp'].value_counts())
#print(TitanicData['Parch'].value_counts())
#print(TitanicData['Embarked'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    object 
 1   Age         891 non-null    float64
 2   SibSp       891 non-null    int64  
 3   Parch       891 non-null    int64  
 4   Fare        891 non-null    float64
 5   Sex_female  891 non-null    uint8  
 6   Sex_male    891 non-null    uint8  
 7   Embarked_C  891 non-null    uint8  
 8   Embarked_Q  891 non-null    uint8  
 9   Embarked_S  891 non-null    uint8  
 10  Pclass_1    891 non-null    uint8  
 11  Pclass_2    891 non-null    uint8  
 12  Pclass_3    891 non-null    uint8  
dtypes: float64(2), int64(2), object(1), uint8(8)
memory usage: 41.9+ KB


## Procesando datos

In [40]:
# convertiremos el campo Survived a codificación one hot, ya que será el valor para nuestra clasificación
# Survived: 0 = No, 1 = Yes
survived = pd.get_dummies(TitanicData['Survived'])
Xinput = TitanicData.drop(['Survived'], axis=1)
Xinput.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Age         891 non-null    float64
 1   SibSp       891 non-null    int64  
 2   Parch       891 non-null    int64  
 3   Fare        891 non-null    float64
 4   Sex_female  891 non-null    uint8  
 5   Sex_male    891 non-null    uint8  
 6   Embarked_C  891 non-null    uint8  
 7   Embarked_Q  891 non-null    uint8  
 8   Embarked_S  891 non-null    uint8  
 9   Pclass_1    891 non-null    uint8  
 10  Pclass_2    891 non-null    uint8  
 11  Pclass_3    891 non-null    uint8  
dtypes: float64(2), int64(2), uint8(8)
memory usage: 34.9 KB


In [41]:
# aplicando la normalización de los datos
Xinput, mediaXinput, stdXinput = normalize(Xinput)
Xinput.head()

,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,-0.592148,0.432550,-0.473408,-0.502163,-0.737281,0.737281,-0.481772,-0.30739,0.615493,-0.565368,-0.509865,0.902081
1,0.638430,0.432550,-0.473408,0.786404,1.354813,-1.354813,2.073341,-0.30739,-1.622891,1.766775,-0.509865,-1.107304
2,-0.284503,-0.474279,-0.473408,-0.488580,1.354813,-1.354813,-0.481772,-0.30739,0.615493,-0.565368,-0.509865,0.902081
3,0.407697,0.432550,-0.473408,0.420494,1.354813,-1.354813,-0.481772,-0.30739,0.615493,1.766775,-0.509865,-1.107304
4,0.407697,-0.474279,-0.473408,-0.486064,-0.737281,0.737281,-0.481772,-0.30739,0.615493,-0.565368,-0.509865,0.902081


In [42]:
print(Xinput.shape)
print(survived.shape)


(891, 12)
(891, 2)


In [43]:
survived


,0,1
0,1,0
1,0,1
2,0,1
3,0,1
4,1,0
...,...,...
886,1,0
887,0,1
888,1,0
889,0,1


In [44]:
surv = survived.iloc[:,1]

In [45]:
regresor_svm = svm.SVR()
regresor_svm.fit(Xinput, surv)

SVR()

In [46]:
Y_pred = regresor_svm.predict(Xinput)


In [47]:
# Error cuadrático medio
print("Mean squared error: %.2f" % mean_squared_error(surv, Y_pred))
# Evaluamos el puntaje de varianza (siendo 1.0 el mejor posible)
print('Variance score: %.2f' % r2_score(surv, Y_pred))

Mean squared error: 0.13
Variance score: 0.47


In [63]:
from sklearn.model_selection import cross_val_score
regre = svm.SVR()
scores = cross_val_score(regre, Xinput, surv, cv=3)
print("Exactitud: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Exactitud: 0.39 (+/- 0.08)


## Kernel lineal

In [57]:
from sklearn.model_selection import cross_val_score
regre = svm.SVC(kernel='linear')
scores = cross_val_score(regre, Xinput, surv, cv=3)
print("Exactitud: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Exactitud: 0.79 (+/- 0.02)


## Kernel Poly

In [61]:
from sklearn.model_selection import cross_val_score
regre = svm.SVC(kernel='poly')
scores = cross_val_score(regre, Xinput, surv, cv=3)
print("Exactitud: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Exactitud: 0.82 (+/- 0.03)


## Kernel RBF

In [62]:
from sklearn.model_selection import cross_val_score
regre = svm.SVC(kernel='rbf')
scores = cross_val_score(regre, Xinput, surv, cv=3)
print("Exactitud: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Exactitud: 0.83 (+/- 0.02)


In [71]:
CL = [ 10**x for x in range(10) ]
CL

[1, 10, 100, 1000, 10000, 100000, 1000000, 10000000, 100000000, 1000000000]

In [72]:
for C in CL:
    regre = svm.SVC(kernel='rbf', C=C)
    scores = cross_val_score(regre, Xinput, surv, cv=3)
    print("Exactitud: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Exactitud: 0.83 (+/- 0.02)
Exactitud: 0.82 (+/- 0.03)
Exactitud: 0.80 (+/- 0.05)
Exactitud: 0.79 (+/- 0.05)
Exactitud: 0.78 (+/- 0.04)
Exactitud: 0.76 (+/- 0.02)
Exactitud: 0.75 (+/- 0.03)
Exactitud: 0.75 (+/- 0.01)
Exactitud: 0.74 (+/- 0.04)
Exactitud: 0.71 (+/- 0.01)
